# Deploy COA Objects in Transcend: DBQL_pivot

This ojbect unpivots the runtime and delaytime metrics that are stored in sperate columns for each time period in DBQL_core. The time period buckets are removed from the column names and converted to rows and a new time_period column.

In [ ]:
%addconnect name=Transcend, user=sc160001, host=tdprdcop3.td.teradata.com, props="LOGMECH=LDAP"

In [8]:
%connect name=Transcend

Password: ·············


Success: 'Transcend' connection established and activated for user 'sc160001'


## Target Table and View (adlste_coa)

In [ ]:
#drop table adlste_coa.coat_dat_DBQL_pivot;

In [ ]:
CREATE MULTISET TABLE adlste_coa.coat_dat_DBQL_pivot ,FALLBACK ,
     NO BEFORE JOURNAL,
     NO AFTER JOURNAL,
     CHECKSUM = DEFAULT,
     DEFAULT MERGEBLOCKRATIO
     (
      Row_Key INTEGER,
      Site_ID VARCHAR(64) CHARACTER SET LATIN NOT CASESPECIFIC,
      LogDate DATE FORMAT 'yyyy-mm-dd',
      LogHour INTEGER,
      LogDateHour TIMESTAMP(0),
      App_Bucket VARCHAR(256) CHARACTER SET LATIN NOT CASESPECIFIC COMPRESS ('BTEQ','JDBC','TPT','Tableau','TPTStream','Python','Unknown','BOBJ','PHP','SQLAssistant','TD-BAR','MS IIS','MS Office','Atanasuite'),
      Use_Bucket VARCHAR(256) CHARACTER SET LATIN NOT CASESPECIFIC COMPRESS ('ETL','Application','Analytic','ETL/Export','Unknown','Backup'),
      Statement_Bucket VARCHAR(256) CHARACTER SET LATIN NOT CASESPECIFIC COMPRESS ('Select','Transaction','Set Session','Insert','Other','Create Table','Delete','Collect Stats','Drop Table','Update','Call SP','Create Object','Merge','Load','Drop Object','Admin','Exec Macro','Unknown'),
      User_Bucket VARCHAR(256) CHARACTER SET LATIN NOT CASESPECIFIC COMPRESS ('Unknown','ETL','TDInternal','Admin','Analytic','TDInternal-DBC'),
      User_Department VARCHAR(256) CHARACTER SET LATIN NOT CASESPECIFIC COMPRESS ('Unknown','dept'),
      User_SubDepartment VARCHAR(256) CHARACTER SET LATIN NOT CASESPECIFIC COMPRESS ('Unknown','subdept'),
      Total_AMPs INTEGER COMPRESS (2928 ,1920 ,144 ),
      time_period VARCHAR(1000) CHARACTER SET UNICODE NOT CASESPECIFIC COMPRESS ('0000_0001','0001_0005','0005_0010','0010_0030','0030_0060','0060_0300','0300_0600','0600_1800','1800_3600','3600_plus'),
      qrycnt_in_runtime BIGINT COMPRESS (0 ,1 ,2 ,3 ,4 ,5 ,6 ,7 ,8 ,9 ,10 ,11 ,12 ,13 ,14 ,15 ,16 ,17 ,18 ,19 ,20 ,21 ,22 ,23 ,24 ,25 ,26 ,27 ,28 ,29 ,30 ,31 ,32 ,33 ,34 ,35 ,36 ,37 ,38 ,39 ,40 ,41 ,42 ,43 ,44 ,45 ,46 ,47 ,48 ,49 ,50 ,51 ,52 ,53 ,54 ,56 ,58 ,60 ,62 ,64 ,66 ,120 ),
      cpusec_in_runtime DECIMAL(18,4) COMPRESS (0.0000 ,0.0040 ),
      iogb_in_runtime DECIMAL(18,4) COMPRESS (0.0000 ,0.0001 ),
      qrycnt_in_delaytime BIGINT COMPRESS (0 ,1 ,2 ,3 ,4 ,6 ),
      cpusec_in_delaytime DECIMAL(18,4) COMPRESS (0.0000 ,0.0040 ),
      iogb_in_delaytime DECIMAL(18,4) COMPRESS (0.0000 ,0.0001 ),
      Run_Id INTEGER DEFAULT -1  COMPRESS 0 )
PRIMARY INDEX ( Site_ID ,LogDateHour );

In [11]:
REPLACE VIEW adlste_coa.coa_dat_DBQL_pivot AS
 LOCK ROW FOR ACCESS
 SELECT 
  Row_Key
  ,Site_ID
  ,LogDate
  ,LogHour
  ,App_Bucket
  ,Use_Bucket
  ,Statement_Bucket
  ,User_Bucket
  ,'n/a' AS Is_Discrete_Human -- remove
  ,User_Department
  ,User_SubDepartment
  ,'n/a' AS User_Region -- remove
  ,'n/a' AS WDName -- remove
  ,'n/a' AS Query_Type -- remove
  ,Total_AMPs
  ,Time_Period
  ,Qrycnt_In_Runtime
  ,CPUsec_In_Runtime
  ,IOgb_In_Runtime
  ,Qrycnt_In_Delaytime
  ,CPUsec_In_Delaytime
  ,IOgb_in_Delaytime
  ,Run_Id
FROM adlste_coa.coat_dat_DBQL_pivot;

Success: 23 rows affected

## Stored Procedure for Full load
This procedure trumcates and reloads all data from the coat_dat_DBQL_core table

In [16]:
REPLACE PROCEDURE adlste_coa.sp_dat_DBQL_pivot_full ()
SQL SECURITY Creator
BEGIN

  /* start setup audit process */
    
  DECLARE runid  INTEGER;
  DECLARE siteid VARCHAR(128);
  DECLARE tablename VARCHAR(128);
  DECLARE callingsp VARCHAR(128); 
  DECLARE startdate DATE;
  DECLARE enddate DATE;
  DECLARE spversion BYTEINT;
  DECLARE SQL_ERROR_CODE INTEGER;
  DECLARE SQL_STATE VARCHAR(5);
  DECLARE ERROR_MESSAGE VARCHAR(256);
  
  DECLARE EXIT HANDLER FOR SqlException
  BEGIN
   
     SET SQL_ERROR_CODE = SqlCode;
     SET SQL_STATE = SqlState;
     
     GET DIAGNOSTICS EXCEPTION 1 ERROR_MESSAGE = Message_Text;
     
     CALL adlste_coa.sp_audit_log(:runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate, 'normal run', 'ERROR');
     
  END;
  
  SET runid = NULL;
  SET tablename = 'adlste_coa.coat_dat_DBQL_pivot';
  SET callingsp = 'adlste_coa.sp_dat_DBQL_pivot_full';
  SET siteid = 'Empty Table';
  SET spversion = 1;
    
  SELECT 
     'ALL'      
    ,Min(LogDate) AS StartDate
    ,Max(LogDate) AS EndDate
  INTO siteid, startdate, enddate
  FROM adlste_coa.coat_dat_DBQL_core 
  GROUP BY 1;
  

    
  /* end setup audit process */
     
  CALL adlste_coa.sp_audit_log(:runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate, 'normal run', 'START');
    
  DELETE FROM adlste_coa.coat_dat_dbql_pivot;

  INSERT INTO adlste_coa.coat_dat_dbql_pivot
  SELECT T1.*
        ,T2.qrycnt_in_delaytime
        ,T2.cpusec_in_delaytime
        ,T2.iogb_in_delaytime
        ,0 AS Run_Id
   FROM 
   (SELECT Row_Key                       
          ,Site_ID                       
          ,LogDate                       
          ,LogHour
          ,LogDateHour          
          ,App_Bucket                    
          ,Use_Bucket
          ,Statement_Bucket
          ,User_Bucket
--          ,Is_Discrete_Human
          ,User_Department
          ,User_SubDepartment
--          ,User_Region
--          ,WDName
--          ,Query_Type
          ,Total_AMPs
          ,time_period
          ,qrycnt_in_runtime
          ,cpusec_in_runtime
          ,iogb_in_runtime
   FROM adlste_coa.coat_dat_dbql_Core UNPIVOT ((qrycnt_in_runtime, cpusec_in_runtime, iogb_in_runtime) FOR time_period
                                      IN ((qrycnt_in_runtime_0000_0001, cpusec_in_runtime_0000_0001, iogb_in_runtime_0000_0001) AS '0 to 1',
                                          (qrycnt_in_runtime_0001_0005, cpusec_in_runtime_0001_0005, iogb_in_runtime_0001_0005) AS '1 to  5',
                                          (qrycnt_in_runtime_0005_0010, cpusec_in_runtime_0005_0010, iogb_in_runtime_0005_0010) AS '5 to 10',
                                          (qrycnt_in_runtime_0010_0030, cpusec_in_runtime_0010_0030, iogb_in_runtime_0010_0030) AS '10 to 30',
                                          (qrycnt_in_runtime_0030_0060, cpusec_in_runtime_0030_0060, iogb_in_runtime_0030_0060) AS '30 to 60',
                                          (qrycnt_in_runtime_0060_0300, cpusec_in_runtime_0060_0300, iogb_in_runtime_0060_0300) AS '60 to 300',
                                          (qrycnt_in_runtime_0300_0600, cpusec_in_runtime_0300_0600, iogb_in_runtime_0300_0600) AS '300 to 600',
                                          (qrycnt_in_runtime_0600_1800, cpusec_in_runtime_0600_1800, iogb_in_runtime_0600_1800) AS '600 to 1800',
                                          (qrycnt_in_runtime_1800_3600, cpusec_in_runtime_1800_3600, iogb_in_runtime_1800_3600) AS '1800 to 3600',
                                          (qrycnt_in_runtime_3600_plus, cpusec_in_runtime_3600_plus, iogb_in_runtime_3600_plus) AS 'over 3600')) T1
   WHERE Row_Key MOD 2 = 0
    ) T1                                 
   JOIN                                           
   (SELECT Row_Key
          ,Site_ID
          ,LogDate
          ,LogHour
          ,((LogDate (FORMAT 'YYYY-MM-DD')) || ' ' || (LogHour (FORMAT '99') (CHAR(2))) || ':00:00') (TIMESTAMP(0)) AS LogDateHour
          ,App_Bucket
          ,Use_Bucket                    
          ,Statement_Bucket
          ,User_Bucket
--          ,Is_Discrete_Human
          ,User_Department
          ,User_SubDepartment
--          ,User_Region
--          ,WDName
--          ,Query_Type
          ,Total_AMPs
          ,time_period
          ,qrycnt_in_delaytime
          ,cpusec_in_delaytime
          ,iogb_in_delaytime
   FROM adlste_coa.coat_dat_dbql_Core UNPIVOT ((qrycnt_in_delaytime, cpusec_in_delaytime, iogb_in_delaytime) FOR time_period
                                      IN ((qrycnt_in_delaytime_0000_0001, cpusec_in_delaytime_0000_0001, iogb_in_delaytime_0000_0001) AS '0 to 1',
                                          (qrycnt_in_delaytime_0001_0005, cpusec_in_delaytime_0001_0005, iogb_in_delaytime_0001_0005) AS '1 to 5',
                                          (qrycnt_in_delaytime_0005_0010, cpusec_in_delaytime_0005_0010, iogb_in_delaytime_0005_0010) AS '5 to 10',
                                          (qrycnt_in_delaytime_0010_0030, cpusec_in_delaytime_0010_0030, iogb_in_delaytime_0010_0030) AS '10 to 30',
                                          (qrycnt_in_delaytime_0030_0060, cpusec_in_delaytime_0030_0060, iogb_in_delaytime_0030_0060) AS '30 to 60',
                                          (qrycnt_in_delaytime_0060_0300, cpusec_in_delaytime_0060_0300, iogb_in_delaytime_0060_0300) AS '60 to 300',
                                          (qrycnt_in_delaytime_0300_0600, cpusec_in_delaytime_0300_0600, iogb_in_delaytime_0300_0600) AS '300 to 600',
                                          (qrycnt_in_delaytime_0600_1800, cpusec_in_delaytime_0600_1800, iogb_in_delaytime_0600_1800) AS '600 to 1800',
                                          (qrycnt_in_delaytime_1800_3600, cpusec_in_delaytime_1800_3600, iogb_in_delaytime_1800_3600) AS '1800 to 3600',
                                          (qrycnt_in_delaytime_3600_plus, cpusec_in_delaytime_3600_plus, iogb_in_delaytime_3600_plus) AS 'over 3600')) T2
   WHERE Row_Key MOD 2 = 0
   ) T2
   ON T1.Site_ID = T2.Site_ID
    AND T1.LogDate = T2.LogDate
    AND T1.LogHour = T2.LogHour
    AND T1.Time_Period = T2.Time_Period
    AND T1.Row_Key = T2.Row_Key

;INSERT INTO adlste_coa.coat_dat_dbql_pivot
  SELECT T1.*
        ,T2.qrycnt_in_delaytime
        ,T2.cpusec_in_delaytime
        ,T2.iogb_in_delaytime 
        ,0 AS Run_Id
   FROM 
   (SELECT Row_Key                       
          ,Site_ID                       
          ,LogDate                       
          ,LogHour
          ,((LogDate (FORMAT 'YYYY-MM-DD')) || ' ' || (LogHour (FORMAT '99') (CHAR(2))) || ':00:00') (TIMESTAMP(0)) AS LogDateHour          
          ,App_Bucket                    
          ,Use_Bucket
          ,Statement_Bucket
          ,User_Bucket
--          ,Is_Discrete_Human
          ,User_Department
          ,User_SubDepartment
--          ,User_Region
--          ,WDName
--          ,Query_Type
          ,Total_AMPs
          ,time_period
          ,qrycnt_in_runtime
          ,cpusec_in_runtime
          ,iogb_in_runtime
   FROM adlste_coa.coat_dat_dbql_Core UNPIVOT ((qrycnt_in_runtime, cpusec_in_runtime, iogb_in_runtime) FOR time_period
                                      IN ((qrycnt_in_runtime_0000_0001, cpusec_in_runtime_0000_0001, iogb_in_runtime_0000_0001) AS '0 to 1',
                                          (qrycnt_in_runtime_0001_0005, cpusec_in_runtime_0001_0005, iogb_in_runtime_0001_0005) AS '1 to  5',
                                          (qrycnt_in_runtime_0005_0010, cpusec_in_runtime_0005_0010, iogb_in_runtime_0005_0010) AS '5 to 10',
                                          (qrycnt_in_runtime_0010_0030, cpusec_in_runtime_0010_0030, iogb_in_runtime_0010_0030) AS '10 to 30',
                                          (qrycnt_in_runtime_0030_0060, cpusec_in_runtime_0030_0060, iogb_in_runtime_0030_0060) AS '30 to 60',
                                          (qrycnt_in_runtime_0060_0300, cpusec_in_runtime_0060_0300, iogb_in_runtime_0060_0300) AS '60 to 300',
                                          (qrycnt_in_runtime_0300_0600, cpusec_in_runtime_0300_0600, iogb_in_runtime_0300_0600) AS '300 to 600',
                                          (qrycnt_in_runtime_0600_1800, cpusec_in_runtime_0600_1800, iogb_in_runtime_0600_1800) AS '600 to 1800',
                                          (qrycnt_in_runtime_1800_3600, cpusec_in_runtime_1800_3600, iogb_in_runtime_1800_3600) AS '1800 to 3600',
                                          (qrycnt_in_runtime_3600_plus, cpusec_in_runtime_3600_plus, iogb_in_runtime_3600_plus) AS 'over 3600')) T1
   WHERE Row_Key MOD 2 = 1
    ) T1                                 
   JOIN                                           
   (SELECT Row_Key
          ,Site_ID
          ,LogDate
          ,LogHour
          ,((LogDate (FORMAT 'YYYY-MM-DD')) || ' ' || (LogHour (FORMAT '99') (CHAR(2))) || ':00:00') (TIMESTAMP(0)) AS LogDateHour          
          ,App_Bucket
          ,Use_Bucket                    
          ,Statement_Bucket
          ,User_Bucket
--          ,Is_Discrete_Human
          ,User_Department
          ,User_SubDepartment
--          ,User_Region
--          ,WDName
--          ,Query_Type
          ,Total_AMPs
          ,time_period
          ,qrycnt_in_delaytime
          ,cpusec_in_delaytime
          ,iogb_in_delaytime
   FROM adlste_coa.coat_dat_dbql_Core UNPIVOT ((qrycnt_in_delaytime, cpusec_in_delaytime, iogb_in_delaytime) FOR time_period
                                      IN ((qrycnt_in_delaytime_0000_0001, cpusec_in_delaytime_0000_0001, iogb_in_delaytime_0000_0001) AS '0 to 1',
                                          (qrycnt_in_delaytime_0001_0005, cpusec_in_delaytime_0001_0005, iogb_in_delaytime_0001_0005) AS '1 to 5',
                                          (qrycnt_in_delaytime_0005_0010, cpusec_in_delaytime_0005_0010, iogb_in_delaytime_0005_0010) AS '5 to 10',
                                          (qrycnt_in_delaytime_0010_0030, cpusec_in_delaytime_0010_0030, iogb_in_delaytime_0010_0030) AS '10 to 30',
                                          (qrycnt_in_delaytime_0030_0060, cpusec_in_delaytime_0030_0060, iogb_in_delaytime_0030_0060) AS '30 to 60',
                                          (qrycnt_in_delaytime_0060_0300, cpusec_in_delaytime_0060_0300, iogb_in_delaytime_0060_0300) AS '60 to 300',
                                          (qrycnt_in_delaytime_0300_0600, cpusec_in_delaytime_0300_0600, iogb_in_delaytime_0300_0600) AS '300 to 600',
                                          (qrycnt_in_delaytime_0600_1800, cpusec_in_delaytime_0600_1800, iogb_in_delaytime_0600_1800) AS '600 to 1800',
                                          (qrycnt_in_delaytime_1800_3600, cpusec_in_delaytime_1800_3600, iogb_in_delaytime_1800_3600) AS '1800 to 3600',
                                          (qrycnt_in_delaytime_3600_plus, cpusec_in_delaytime_3600_plus, iogb_in_delaytime_3600_plus) AS 'over 3600')) T2
   WHERE Row_Key MOD 2 = 1
   ) T2
   ON T1.Site_ID = T2.Site_ID
    AND T1.LogDate = T2.LogDate
    AND T1.LogHour = T2.LogHour
    AND T1.Time_Period = T2.Time_Period
    AND T1.Row_Key = T2.Row_Key;
 
  CALL adlste_coa.sp_audit_log(:runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate, 'normal run', 'END');
END;

## Stored Procedure for delta loads
This procedure performan and incremental load of the data from the adlste_coa_stg.stg_dat_DBQL_core staging table

In [ ]:
REPLACE PROCEDURE adlste_coa.sp_dat_DBQL_pivot_delta ()
SQL SECURITY Creator
BEGIN

  /* start setup audit process */
    
  DECLARE runid  INTEGER;
  DECLARE siteid VARCHAR(128);
  DECLARE tablename VARCHAR(128);
  DECLARE callingsp VARCHAR(128); 
  DECLARE startdate DATE;
  DECLARE enddate DATE;
  DECLARE spversion BYTEINT;
  DECLARE SQL_ERROR_CODE INTEGER;
  DECLARE SQL_STATE VARCHAR(5);
  DECLARE ERROR_MESSAGE VARCHAR(256);
  
  DECLARE EXIT HANDLER FOR SqlException
  BEGIN
   
     SET SQL_ERROR_CODE = SqlCode;
     SET SQL_STATE = SqlState;
     
     GET DIAGNOSTICS EXCEPTION 1 ERROR_MESSAGE = Message_Text;
     
     CALL adlste_coa.sp_audit_log(:runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate, 'normal run', 'ERROR');
     
  END;
  
  SET runid = NULL;
  SET tablename = 'adlste_coa.coat_dat_DBQL_pivot';
  SET callingsp = 'adlste_coa.sp_dat_DBQL_pivot_delta';
  SET siteid = 'Empty Table';
  SET spversion = 1;
    
  SELECT 
     Site_Id  
    ,Min(LogDate) AS StartDate
    ,Max(LogDate) AS EndDate
  INTO siteid, startdate, enddate
  FROM adlste_coa_stg.stg_dat_DBQL_core 
  GROUP BY 1;
    
  /* end setup audit process */
     
  CALL adlste_coa.sp_audit_log(:runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate, 'normal run', 'START');
       
  DELETE FROM adlste_coa.coat_dat_dbql_pivot
  WHERE (Site_Id, LogDate) IN (SELECT Site_Id, LogDate FROM adlste_coa_stg.stg_dat_DBQL_core);

  INSERT INTO adlste_coa.coat_dat_dbql_pivot
  SELECT T1.*
        ,T2.qrycnt_in_delaytime
        ,T2.cpusec_in_delaytime
        ,T2.iogb_in_delaytime
        ,0 AS Run_Id
   FROM 
   (SELECT Row_Key                       
          ,Site_ID                       
          ,LogDate                       
          ,LogHour                       
          ,App_Bucket                    
          ,Use_Bucket
          ,Statement_Bucket
          ,User_Bucket
          ,Is_Discrete_Human
          ,User_Department
          ,User_SubDepartment
          ,User_Region
          ,WDName
          ,Query_Type
          ,Total_AMPs
          ,time_period
          ,qrycnt_in_runtime
          ,cpusec_in_runtime
          ,iogb_in_runtime
   FROM adlste_coa_stg.stg_dat_DBQL_core UNPIVOT ((qrycnt_in_runtime, cpusec_in_runtime, iogb_in_runtime) FOR time_period
                                      IN ((qrycnt_in_runtime_0000_0001, cpusec_in_runtime_0000_0001, iogb_in_runtime_0000_0001) AS '0 to 1',
                                          (qrycnt_in_runtime_0001_0005, cpusec_in_runtime_0001_0005, iogb_in_runtime_0001_0005) AS '1 to  5',
                                          (qrycnt_in_runtime_0005_0010, cpusec_in_runtime_0005_0010, iogb_in_runtime_0005_0010) AS '5 to 10',
                                          (qrycnt_in_runtime_0010_0030, cpusec_in_runtime_0010_0030, iogb_in_runtime_0010_0030) AS '10 to 30',
                                          (qrycnt_in_runtime_0030_0060, cpusec_in_runtime_0030_0060, iogb_in_runtime_0030_0060) AS '30 to 60',
                                          (qrycnt_in_runtime_0060_0300, cpusec_in_runtime_0060_0300, iogb_in_runtime_0060_0300) AS '60 to 300',
                                          (qrycnt_in_runtime_0300_0600, cpusec_in_runtime_0300_0600, iogb_in_runtime_0300_0600) AS '300 to 600',
                                          (qrycnt_in_runtime_0600_1800, cpusec_in_runtime_0600_1800, iogb_in_runtime_0600_1800) AS '600 to 1800',
                                          (qrycnt_in_runtime_1800_3600, cpusec_in_runtime_1800_3600, iogb_in_runtime_1800_3600) AS '1800 to 3600',
                                          (qrycnt_in_runtime_3600_plus, cpusec_in_runtime_3600_plus, iogb_in_runtime_3600_plus) AS 'over 3600')) T1
   WHERE Row_Key MOD 2 = 0
    ) T1                                 
   JOIN                                           
   (SELECT Row_Key
          ,Site_ID
          ,LogDate
          ,LogHour
          ,App_Bucket
          ,Use_Bucket                    
          ,Statement_Bucket
          ,User_Bucket
          ,Is_Discrete_Human
          ,User_Department
          ,User_SubDepartment
          ,User_Region
          ,WDName
          ,Query_Type
          ,Total_AMPs
          ,time_period
          ,qrycnt_in_delaytime
          ,cpusec_in_delaytime
          ,iogb_in_delaytime
   FROM adlste_coa_stg.stg_dat_DBQL_core UNPIVOT ((qrycnt_in_delaytime, cpusec_in_delaytime, iogb_in_delaytime) FOR time_period
                                      IN ((qrycnt_in_delaytime_0000_0001, cpusec_in_delaytime_0000_0001, iogb_in_delaytime_0000_0001) AS '0 to 1',
                                          (qrycnt_in_delaytime_0001_0005, cpusec_in_delaytime_0001_0005, iogb_in_delaytime_0001_0005) AS '1 to 5',
                                          (qrycnt_in_delaytime_0005_0010, cpusec_in_delaytime_0005_0010, iogb_in_delaytime_0005_0010) AS '5 to 10',
                                          (qrycnt_in_delaytime_0010_0030, cpusec_in_delaytime_0010_0030, iogb_in_delaytime_0010_0030) AS '10 to 30',
                                          (qrycnt_in_delaytime_0030_0060, cpusec_in_delaytime_0030_0060, iogb_in_delaytime_0030_0060) AS '30 to 60',
                                          (qrycnt_in_delaytime_0060_0300, cpusec_in_delaytime_0060_0300, iogb_in_delaytime_0060_0300) AS '60 to 300',
                                          (qrycnt_in_delaytime_0300_0600, cpusec_in_delaytime_0300_0600, iogb_in_delaytime_0300_0600) AS '300 to 600',
                                          (qrycnt_in_delaytime_0600_1800, cpusec_in_delaytime_0600_1800, iogb_in_delaytime_0600_1800) AS '600 to 1800',
                                          (qrycnt_in_delaytime_1800_3600, cpusec_in_delaytime_1800_3600, iogb_in_delaytime_1800_3600) AS '1800 to 3600',
                                          (qrycnt_in_delaytime_3600_plus, cpusec_in_delaytime_3600_plus, iogb_in_delaytime_3600_plus) AS 'over 3600')) T2
   WHERE Row_Key MOD 2 = 0
   ) T2
   ON T1.Site_ID = T2.Site_ID
    AND T1.LogDate = T2.LogDate
    AND T1.LogHour = T2.LogHour
    AND T1.Time_Period = T2.Time_Period
    AND T1.Row_Key = T2.Row_Key

;INSERT INTO adlste_coa.coat_dat_dbql_pivot
  SELECT T1.*
        ,T2.qrycnt_in_delaytime
        ,T2.cpusec_in_delaytime
        ,T2.iogb_in_delaytime 
        ,0 AS Run_Id
   FROM 
   (SELECT Row_Key                       
          ,Site_ID                       
          ,LogDate                       
          ,LogHour                       
          ,App_Bucket                    
          ,Use_Bucket
          ,Statement_Bucket
          ,User_Bucket
          ,Is_Discrete_Human
          ,User_Department
          ,User_SubDepartment
          ,User_Region
          ,WDName
          ,Query_Type
          ,Total_AMPs
          ,time_period
          ,qrycnt_in_runtime
          ,cpusec_in_runtime
          ,iogb_in_runtime
   FROM adlste_coa_stg.stg_dat_DBQL_core UNPIVOT ((qrycnt_in_runtime, cpusec_in_runtime, iogb_in_runtime) FOR time_period
                                      IN ((qrycnt_in_runtime_0000_0001, cpusec_in_runtime_0000_0001, iogb_in_runtime_0000_0001) AS '0 to 1',
                                          (qrycnt_in_runtime_0001_0005, cpusec_in_runtime_0001_0005, iogb_in_runtime_0001_0005) AS '1 to  5',
                                          (qrycnt_in_runtime_0005_0010, cpusec_in_runtime_0005_0010, iogb_in_runtime_0005_0010) AS '5 to 10',
                                          (qrycnt_in_runtime_0010_0030, cpusec_in_runtime_0010_0030, iogb_in_runtime_0010_0030) AS '10 to 30',
                                          (qrycnt_in_runtime_0030_0060, cpusec_in_runtime_0030_0060, iogb_in_runtime_0030_0060) AS '30 to 60',
                                          (qrycnt_in_runtime_0060_0300, cpusec_in_runtime_0060_0300, iogb_in_runtime_0060_0300) AS '60 to 300',
                                          (qrycnt_in_runtime_0300_0600, cpusec_in_runtime_0300_0600, iogb_in_runtime_0300_0600) AS '300 to 600',
                                          (qrycnt_in_runtime_0600_1800, cpusec_in_runtime_0600_1800, iogb_in_runtime_0600_1800) AS '600 to 1800',
                                          (qrycnt_in_runtime_1800_3600, cpusec_in_runtime_1800_3600, iogb_in_runtime_1800_3600) AS '1800 to 3600',
                                          (qrycnt_in_runtime_3600_plus, cpusec_in_runtime_3600_plus, iogb_in_runtime_3600_plus) AS 'over 3600')) T1
   WHERE Row_Key MOD 2 = 1
    ) T1                                 
   JOIN                                           
   (SELECT Row_Key
          ,Site_ID
          ,LogDate
          ,LogHour
          ,App_Bucket
          ,Use_Bucket                    
          ,Statement_Bucket
          ,User_Bucket
          ,Is_Discrete_Human
          ,User_Department
          ,User_SubDepartment
          ,User_Region
          ,WDName
          ,Query_Type
          ,Total_AMPs
          ,time_period
          ,qrycnt_in_delaytime
          ,cpusec_in_delaytime
          ,iogb_in_delaytime
   FROM adlste_coa_stg.stg_dat_DBQL_core UNPIVOT ((qrycnt_in_delaytime, cpusec_in_delaytime, iogb_in_delaytime) FOR time_period
                                      IN ((qrycnt_in_delaytime_0000_0001, cpusec_in_delaytime_0000_0001, iogb_in_delaytime_0000_0001) AS '0 to 1',
                                          (qrycnt_in_delaytime_0001_0005, cpusec_in_delaytime_0001_0005, iogb_in_delaytime_0001_0005) AS '1 to 5',
                                          (qrycnt_in_delaytime_0005_0010, cpusec_in_delaytime_0005_0010, iogb_in_delaytime_0005_0010) AS '5 to 10',
                                          (qrycnt_in_delaytime_0010_0030, cpusec_in_delaytime_0010_0030, iogb_in_delaytime_0010_0030) AS '10 to 30',
                                          (qrycnt_in_delaytime_0030_0060, cpusec_in_delaytime_0030_0060, iogb_in_delaytime_0030_0060) AS '30 to 60',
                                          (qrycnt_in_delaytime_0060_0300, cpusec_in_delaytime_0060_0300, iogb_in_delaytime_0060_0300) AS '60 to 300',
                                          (qrycnt_in_delaytime_0300_0600, cpusec_in_delaytime_0300_0600, iogb_in_delaytime_0300_0600) AS '300 to 600',
                                          (qrycnt_in_delaytime_0600_1800, cpusec_in_delaytime_0600_1800, iogb_in_delaytime_0600_1800) AS '600 to 1800',
                                          (qrycnt_in_delaytime_1800_3600, cpusec_in_delaytime_1800_3600, iogb_in_delaytime_1800_3600) AS '1800 to 3600',
                                          (qrycnt_in_delaytime_3600_plus, cpusec_in_delaytime_3600_plus, iogb_in_delaytime_3600_plus) AS 'over 3600')) T2
   WHERE Row_Key MOD 2 = 1
   ) T2
   ON T1.Site_ID = T2.Site_ID
    AND T1.LogDate = T2.LogDate
    AND T1.LogHour = T2.LogHour
    AND T1.Time_Period = T2.Time_Period
    AND T1.Row_Key = T2.Row_Key;

   CALL adlste_coa.sp_audit_log(:runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate, 'normal run', 'END');
  
END;

## Execute the full load procedure

In [17]:
call adlste_coa.sp_dat_DBQL_pivot_full();

ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:3812:SP_DAT_DBQL_PIVOT_FULL:The positional assignment list has too few values.

## Report Success

In [ ]:
SELECT * 
FROM adlste_coa.coa_audit_summary
WHERE LogDate = DATE
  AND CallingSP = 'adlste_coa.sp_dat_DBQL_full'
ORDER BY 1 DESC